In [5]:
# import pandas as pd


# def datenLaden():
#     # load data columnsName in row 4

#     df = pd.read_excel("MLGT.xlsx",header=3) 
#     return df
# df= datenLaden()
# #df.fillna(0)
# df

,Unnamed: 0,MANDT,MATNR,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,LGNUM,LGTYP,LVORM,LGPLA,LPMAX,Unnamed: 12,Unnamed: 13,LPMIN,Unnamed: 15,MAMNG,NSMNG,KOBER,RDMNG
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,170.0,10081810.0,NaN,NaN,NaN,NaN,DE2,SN1,NaN,NaN,165.75,NaN,NaN,153.0,NaN,152.999,4.25,NaN,4.25
2,NaN,170.0,10081810.0,NaN,NaN,NaN,NaN,DE2,TN1,NaN,NaN,17.00,NaN,NaN,8.5,NaN,4.249,8.50,NaN,0.17
3,NaN,170.0,10081878.0,NaN,NaN,NaN,NaN,DE2,SN1,NaN,NaN,165.75,NaN,NaN,153.0,NaN,152.999,4.25,NaN,4.25
4,NaN,170.0,10081878.0,NaN,NaN,NaN,NaN,DE2,TN1,NaN,NaN,17.00,NaN,NaN,8.5,NaN,4.249,8.50,NaN,0.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2557,NaN,170.0,50063231.0,NaN,NaN,NaN,NaN,DE2,TN1,NaN,R-12-01-61,40.00,NaN,NaN,20.0,NaN,9.999,20.00,NaN,1.00
2558,NaN,170.0,50063261.0,NaN,NaN,NaN,NaN,DE2,SN1,NaN,R05-15,230.00,NaN,NaN,200.0,NaN,199.999,10.00,NaN,10.00
2559,NaN,170.0,50063261.0,NaN,NaN,NaN,NaN,DE2,TN1,NaN,R-12-01-62,40.00,NaN,NaN,20.0,NaN,9.999,20.00,NaN,1.00
2560,NaN,170.0,50063290.0,NaN,NaN,NaN,NaN,DE2,SN1,NaN,R04-22,230.00,NaN,NaN,200.0,NaN,199.999,10.00,NaN,10.00


In [2]:



import pandas as pd

# create the first DataFrame
df1 = pd.DataFrame({"A": ["A1", "A2", "A3"], "B": ["B1", "B2", "B3"], "C": ["C1", "C2", "C3"]})

# create the second DataFrame
df2 = pd.DataFrame({"B": ["B1", "B2", "B3"], "C": ["C1", "C2", "C3"], "D": ["D1", "D2", "D3"]})

# merge the two DataFrames, using the 'B' and 'C' columns as the keys, and only include columns 'A', 'C', and 'D'
df3 = df1.merge(df2, on=["B", "C"], how="inner", subset=["A", "C", "D"])

# print the resulting DataFrame
print(df3)


TypeError: DataFrame.merge() got an unexpected keyword argument 'subset'